In [24]:
# Tạo cấu trúc thư mục chuẩn
!mkdir -p rag_langchain/data_source/generative_ai
!mkdir -p rag_langchain/src/base
!mkdir -p rag_langchain/src/rag


In [25]:
%%writefile rag_langchain/data_source/generative_ai/download.py
import os
import wget

file_links = [
    {"title": "Attention Is All You Need", "url": "https://arxiv.org/pdf/1706.03762"},
    {"title": "BERT - Pre-training of Deep Bidirectional Transformers for Language Understanding", "url": "https://arxiv.org/pdf/1810.04805"},
    {"title": "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models", "url": "https://arxiv.org/pdf/2201.11903"},
    {"title": "Denoising Diffusion Probabilistic Models", "url": "https://arxiv.org/pdf/2006.11239"},
    {"title": "Instruction Tuning for Large Language Models - A Survey", "url": "https://arxiv.org/pdf/2308.10792"},
    {"title": "Llama 2- Open Foundation and Fine-Tuned Chat Models", "url": "https://arxiv.org/pdf/2307.09288"}
]

def is_exist(file_link):
    return os.path.exists(f"./{file_link['title']}.pdf")

for file_link in file_links:
    if not is_exist(file_link):
        wget.download(file_link["url"], out=f"./{file_link['title']}.pdf")


Overwriting rag_langchain/data_source/generative_ai/download.py


In [26]:
%%writefile rag_langchain/src/base/llm_model.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

# Cấu hình quantization cho mô hình
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

def get_hf_llm(model_name: str = "meta-llama/Llama-3-8B-Instruct", max_new_token=1024):
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_token,
        pad_token_id=tokenizer.eos_token_id,
        device_map="auto"
    )

    llm = HuggingFacePipeline(
        pipeline=model_pipeline
    )

    return llm


Overwriting rag_langchain/src/base/llm_model.py


In [27]:
%%writefile rag_langchain/src/rag/file_loader.py
from typing import Union, List, Literal
import glob
from tqdm import tqdm
import multiprocessing
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Hàm để loại bỏ các ký tự không phải UTF-8
def remove_non_utf8_characters(text):
    return ''.join(char for char in text if ord(char) < 128)

# Hàm load PDF
def load_pdf(pdf_file):
    docs = PyPDFLoader(pdf_file, extract_images=True).load()
    for doc in docs:
        doc.page_content = remove_non_utf8_characters(doc.page_content)
    return docs

# Hàm lấy số CPU
def get_num_cpu():
    return multiprocessing.cpu_count()

# Base loader class
class BaseLoader:
    def __init__(self) -> None:
        self.num_processes = get_num_cpu()

    def __call__(self, files: List[str], **kwargs):
        pass

# PDF loader class
class PDFLoader(BaseLoader):
    def __init__(self) -> None:
        super().__init__()

    def __call__(self, pdf_files: List[str], **kwargs):
        num_processes = min(self.num_processes, kwargs["workers"])
        with multiprocessing.Pool(processes=num_processes) as pool:
            doc_loaded = []
            total_files = len(pdf_files)
            with tqdm(total=total_files, desc="Loading PDFs", unit="file") as pbar:
                for result in pool.imap_unordered(load_pdf, pdf_files):
                    doc_loaded.extend(result)
                    pbar.update(1)
        return doc_loaded

# TextSplitter class
class TextSplitter:
    def __init__(self, separators: List[str] = ['\n\n', '\n', ' ', ''], chunk_size: int = 300, chunk_overlap: int = 0) -> None:
        self.splitter = RecursiveCharacterTextSplitter(
            separators=separators,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
        )

    def __call__(self, documents):
        return self.splitter.split_documents(documents)

# Loader class
class Loader:
    def __init__(self, file_type: str = Literal["pdf"], split_kwargs: dict = {"chunk_size": 300, "chunk_overlap": 0}) -> None:
        assert file_type in ["pdf"], "file_type must be pdf"
        self.file_type = file_type
        if file_type == "pdf":
            self.doc_loader = PDFLoader()
        else:
            raise ValueError("file_type must be pdf")

        self.doc_spltter = TextSplitter(**split_kwargs)

    def load(self, pdf_files: Union[str, List[str]], workers: int = 1):
        if isinstance(pdf_files, str):
            pdf_files = [pdf_files]
        doc_loaded = self.doc_loader(pdf_files, workers=workers)
        doc_split = self.doc_spltter(doc_loaded)
        return doc_split

    def load_dir(self, dir_path: str, workers: int = 1):
        if self.file_type == "pdf":
            files = glob.glob(f"{dir_path}/*.pdf")
            assert len(files) > 0, f"No {self.file_type} files found in {dir_path}"
        else:
            raise ValueError("file_type must be pdf")

        return self.load(files, workers=workers)


Overwriting rag_langchain/src/rag/file_loader.py


In [28]:
%%writefile rag_langchain/src/rag/vectorstore.py
from typing import Union
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

class VectorDB:
    def __init__(self,
                 documents=None,
                 vector_db: Union[Chroma, FAISS] = Chroma,
                 embedding=HuggingFaceEmbeddings()) -> None:
        self.vector_db = vector_db
        self.embedding = embedding
        self.db = self._build_db(documents)

    def _build_db(self, documents):
        db = self.vector_db.from_documents(documents=documents, embedding=self.embedding)
        return db

    def get_retriever(self, search_type: str = "similarity", search_kwargs: dict = {"k": 10}):
        retriever = self.db.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
        return retriever


Overwriting rag_langchain/src/rag/vectorstore.py


In [29]:
%%writefile rag_langchain/src/rag/offline_rag.py
import re
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

class Str_OutputParser(StrOutputParser):
    def __init__(self) -> None:
        super().__init__()

    def parse(self, text: str) -> str:
        return self.extract_answer(text)

    def extract_answer(self,
                       text_response: str,
                       pattern: str = r"Answer :\s*(.*)") -> str:
        match = re.search(pattern, text_response, re.DOTALL)
        if match:
            answer_text = match.group(1).strip()
            return answer_text
        else:
            return text_response

class Offline_RAG:
    def __init__(self, llm) -> None:
        self.llm = llm
        self.prompt = hub.pull("rlm/rag-prompt")
        self.str_parser = Str_OutputParser()

    def get_chain(self, retriever):
        input_data = {
            "context": retriever | self.format_docs,
            "question": RunnablePassthrough()
        }
        rag_chain = (
            input_data
            | self.prompt
            | self.llm
            | self.str_parser
        )
        return rag_chain

    def format_docs(self, docs):
        return "\n\n".join(doc.page_content for doc in docs)


Overwriting rag_langchain/src/rag/offline_rag.py


In [30]:
%%writefile rag_langchain/src/rag/utils.py
import re

def extract_answer(text_response: str, pattern: str = r"Answer :\s*(.*)") -> str:
    match = re.search(pattern, text_response)
    if match:
        answer_text = match.group(1).strip()
        return answer_text
    else:
        return "Answer not found."


Overwriting rag_langchain/src/rag/utils.py


In [31]:
%%writefile rag_langchain/src/rag/main.py
from pydantic import BaseModel, Field

from src.rag.file_loader import Loader
from src.rag.vectorstore import VectorDB
from src.rag.offline_rag import Offline_RAG

# Lớp InputQA dùng để xác định câu hỏi người dùng
class InputQA(BaseModel):
    question: str = Field(..., title="Question to ask the model")

# Lớp OutputQA dùng để xác định câu trả lời từ mô hình
class OutputQA(BaseModel):
    answer: str = Field(..., title="Answer from the model")

# Hàm build_rag_chain khởi tạo toàn bộ quy trình RAG
def build_rag_chain(llm, data_dir, data_type):
    # Tải dữ liệu và chia nhỏ thành các tài liệu
    doc_loaded = Loader(file_type=data_type).load_dir(data_dir, workers=2)

    # Tạo cơ sở dữ liệu vector và retriever
    retriever = VectorDB(documents=doc_loaded).get_retriever()

    # Xây dựng RAG chain
    rag_chain = Offline_RAG(llm).get_chain(retriever)

    return rag_chain


Overwriting rag_langchain/src/rag/main.py


In [32]:
%%writefile rag_langchain/src/app.py
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

from langserve import add_routes
from src.base.llm_model import get_hf_llm
from src.rag.main import build_rag_chain, InputQA, OutputQA

# Khởi tạo mô hình LLM
llm = get_hf_llm(temperature=0.9)

# Đường dẫn tới tài liệu
genai_docs = "./data_source/generative_ai"

# --------- Chains -------------------
genai_chain = build_rag_chain(llm, data_dir=genai_docs, data_type="pdf")

# --------- App - FastAPI ----------------
app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple api server using Langchain’s Runnable interfaces",
)

# Cấu hình CORS cho phép truy cập từ tất cả các nguồn
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"],
)

# --------- Routes - FastAPI ----------------

# Endpoint kiểm tra trạng thái server
@app.get("/check")
async def check():
    return {"status": "ok"}

# Endpoint xử lý câu hỏi và trả về câu trả lời
@app.post("/generative_ai", response_model=OutputQA)
async def generative_ai(inputs: InputQA):
    answer = genai_chain.invoke(inputs.question)
    return {"answer": answer}

# --------- Langserve Routes - Playground ----------------
add_routes(
    app,
    genai_chain,
    playground_type="default",
    path="/generative_ai"
)


Overwriting rag_langchain/src/app.py


In [33]:
%%writefile rag_langchain/requirements.txt
torch==2.2.2
transformers==4.39.3
accelerate==0.28.0
bitsandbytes==0.42.0
huggingface-hub==0.22.2
langchain==0.1.14
langchain-core==0.1.43
langchain-community==0.0.31
pypdf==4.2.0
sentence-transformers==2.6.1
beautifulsoup4==4.12.3
langserve[all]
chromadb==0.4.24
langchain-chroma==0.1.0
faiss-cpu==1.8.0
unstructured==0.13.2
fastapi==0.110.1
uvicorn==0.29.0


Overwriting rag_langchain/requirements.txt


In [34]:
!pip install uvicorn
!pip install pyngrok


In [35]:
from pyngrok import ngrok

# Thêm authtoken vào ngrok
ngrok.set_auth_token("2w4qyWEnLQ7JNLh8Ck2cS6Nu927_hNKdLKSL1tSSaGR6VjZY")

# Kiểm tra kết nối
public_url = ngrok.connect(5000)
print("API is live at:", public_url)


API is live at: NgrokTunnel: "https://59b2-34-124-176-98.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
import subprocess

# Khởi động server uvicorn trên cổng 5000
subprocess.run(["uvicorn", "src.app:app", "--host", "0.0.0.0", "--port", "5000", "--reload"])
